In [1]:
import sys
quic_path = '/home/quic/QUIC-Projects'
if not quic_path in sys.path:
    sys.path.append(quic_path)

In [2]:
import numpy as np 
from matplotlib import pyplot as plt

In [3]:
from classifiers.quantum.qasvm import QASVM
from classifiers.quantum import Qasvm_Mapping_4x2
from classifiers.optimizer import MySPSA
from classifiers.optimizer import SimpleStorage
from qiskit.circuit.library import RealAmplitudes
from qiskit.aqua import QuantumInstance
from qiskit.providers.aer import QasmSimulator
from qiskit.ignis.mitigation import CompleteMeasFitter
from qiskit import IBMQ


ImportError: cannot import name 'SimpleStorage' from 'classifiers.optimizer' (/home/quic/QUIC-Projects/classifiers/optimizer.py)

In [ ]:
import torch
from torch.autograd import Function
from torch import optim
from torch import nn
from torch.nn import functional as F

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-kaist', group='internal', project='default')
backend = provider.get_backend('ibmq_toronto')
backend

In [ ]:
X =np.array([[ 0.72294659, -1.00386432],
            [-0.60553577,  2.29966755],
            [-2.50699176, -1.03101898],
            [ 2.63961761,  2.21632328]])
y = np.array([0,0,1,1])
var_form = RealAmplitudes(2, reps=1)
storage = SimpleStorage()
optimizer = MySPSA(maxiter=2**10, last_avg=2**7, callback=storage)
quantum_instance = QuantumInstance(backend=QasmSimulator.from_backend(backend), shots=2**13, seed_transpiler=0, measurement_error_mitigation_cls=CompleteMeasFitter, initial_layout=Qasvm_Mapping_4x2(backend).layout, optimization_level=2)
quantum_instance.backend_config['basis_gates'] = backend.configuration().basis_gates
qasvm = QASVM(X, y, var_form=var_form, optimizer=optimizer, quantum_instance=quantum_instance, C=1, k=0.1, option='Bloch_sphere').dual
qasvm

In [ ]:
class TensorQASVM(Function):

    @staticmethod
    def forward(ctx, input, qasvm):
        ctx.qasvm = qasvm
        evaluation = torch.tensor([ctx.qasvm.cost_fn(input.tolist())])
        ctx.save_for_backward(input)
        return evaluation

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors

        if ctx.qasvm.grad_fn is not None:
            gradients = ctx.qasvm.grad_fn(input.tolist())
        else:
            input_list = np.array(input.tolist())
            input_list_rights = input_list + 1e-3*np.eye(len(input_list))
            input_list_lefts = input_list + 1e-3*np.eye(len(input_list))

            gradients = (np.array(list(map(ctx.qasvm.cost_fn, input_list_rights))) - np.array(list(map(ctx.qasvm.cost_fn, input_list_lefts))))/(2*1e-3)

        return grad_output * gradients, None

In [ ]:
class QASVM_layer(nn.Module):
    def __init__(self, qasvm):
        super().__init__()
        self.qasvm = qasvm
        self.thetas = nn.Parameter(torch.randn(qasvm.num_parameters))
    def forward(self):
        return TensorQASVM.apply(self.thetas, self.qasvm)


In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(comment = 'SGD qasvm')
model = QASVM_layer(qasvm)


In [ ]:
from tqdm.notebook import tqdm
optimizer = optim.SGD(model.parameters(), lr=0.01, )

epochs = 2**10

model.train()
obj_vals = []
params = []
param_names = [f'param_{i}' for i in range(qasvm.num_parameters)]
for epoch in tqdm(range(epochs)):
    optimizer.zero_grad()
    output = model()
    output.backward()
    optimizer.step()
    with torch.no_grad():
        obj_vals.append(output.item()) 
        writer.add_scalar('Train/objectiv_value', output, epoch)
        thetas = model.thetas.data
        params.append(thetas)
        writer.add_scalars('Parameters', dict(zip(param_names, thetas)), epoch)

writer.close()

In [ ]:
param_list = model.thetas.data.numpy()

In [ ]:
from qiskit.aqua.algorithms import VQResult
result = VQResult()
result.optimizer_evals = 1000
result.optimizer_time = 47*60+57
result.optimal_value = qasvm.cost_fn(param_list)
result.optimal_point = param_list
result.optimal_parameters = dict(zip(qasvm.var_form_params['0'], param_list))
qasvm.result = result

In [ ]:
'''
params = np.array(list(storage.params.values()))
steps = np.array(list(storage.params.keys()))
[plt.plot(steps, params[:,i], label=f'parameter {i}') for i in range(len(qasvm.var_form_params['0']))]
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlim([min(steps), max(steps)])
plt.yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi], ['-$\pi$', '-$\pi/2$', '$0$', '$\pi/2$', '$\pi$'])
plt.xlabel('steps')
plt.ylabel('param value')
plt.grid()'''

In [ ]:
from classifiers.convex.svm import BinarySVM
from classifiers.kernel import Kernel
kernel = Kernel('SingleQubit')
svm = BinarySVM(kernel, C=None, k=0.1, mutation='uniform_QASVM')
svm.fit(X, y)
svm.accuracy(X, y)

In [ ]:
plt.plot(svm.f(svm.data), label='sim')
res = np.array([qasvm.f(qasvm.data) for _ in range(10)])
plt.errorbar(range(len(res.mean(axis=0))), res.mean(axis=0), yerr=2*res.std(axis=0), label='qasvm')
plt.xticks(range(4), [f'Data {i}' for i in range(4)])
plt.ylabel('f')
plt.legend()
plt.grid()